# Logger Handler Widget for Jupyter-based Gateway Tools
> An example of a log handler widget used to replace intrusive Jupyter cell output. 

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- published: false

Jupyter notebooks are increasingly being used to develop tools on science gateways because they are interactive, easy to deploy, and portable. The developer may want a way to capture logs in a way that doesn't cause additional cell output. The first task is to inhibit such output.

In [4]:
import warnings
warnings.warn("Not caputured")

<ipython-input-4-e8947fda65c4>:2: UserWarning: Not caputured
  warnings.warn("Not caputured")


In [5]:
import logging
logging.captureWarnings(True)

In [7]:
warnings.warn("Caputured!")

The next step is to create a widget designed to handle, and thereby display, these captured logs.

In [11]:
# log.py
import logging
import ipywidgets as ipyw
from IPython.display import Javascript

class LogHandler(logging.Handler): 
    
    def __init__(self):
        super().__init__()
        layout = {'width': '100%', 'height': '250px'}
        self.widget = ipyw.Textarea(value='', layout=layout)
        self.widget.disabled=True
        self.widget.add_class('loggingHandler')

    def emit(self, record):
        ''' Overload of logging.Handler method '''
        formatted_record = self.format(record)
        self.widget.value += str(formatted_record) + '\n'     
        
    def display(self):
        display(self.widget)
           
    def createLogger(self, name):
        logger = logging.getLogger(name)
        logger.setLevel(LEVEL)
        logger.addHandler(self)
        return logger
    
LEVEL = logging.INFO
FORMATTER = logging.Formatter('%(name)s:%(lineno)d: %(message)s')
HANDLER = LogHandler()
HANDLER.setFormatter(FORMATTER)
logger = HANDLER.createLogger(__name__)
logger.info('Log handler setup complete.')

In [12]:
HANDLER

<LogHandler (NOTSET)>

In [13]:
# interface.py
import os
from IPython.display import Javascript, HTML

loggingHandler = HANDLER.widget
logger = HANDLER.createLogger(__name__)

class Interface:
    ''' Interface contains two major app components: Four main tabs and output log accordion '''
    FILE_BROWSER_COLS = ['filename','size','date/time']

    def __init__(self):

        
        ''' Add LogHandler.logTextarea widget to LogAccordian and display '''
        logAccordion = ipyw.Accordion()
        logAccordion.set_title(0, 'Logs')
        logAccordion.children = (loggingHandler,)
        logAccordion.add_class('logAccordion')
        display(logAccordion)
        logger.info('Initializing GUI...')
        
        # other app components
        '''
        main = ipyw.Tab( children = ( ipyw.IntSlider(
                value=7,
                min=0,
                max=10,
                step=1,
                description='Test:',
                disabled=False,
            ),)
        )
        main.set_title(0, 'My Application')
        display(main)
        '''
        html = '''
        <script data-main="/js/config.js" src="js/require.js"></script>
        <script>
        //alert(window.location.pathname);
        require(['config'], function() {
            alert("bar")
        })
        </script>
        '''
        display(HTML(html))
        
    

In [14]:
# app.py
display(Javascript('js/require.js'))
display(Javascript('''
require.config({
    baseUrl: 
})
'''))

app = Interface()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Accordion(children=(Textarea(value='__main__:34: Log handler setup complete.\n', disabled=True, layout=Layout(…

In [15]:
%%js
// Make logAccordion be the last output_area in the output div
// Note: we renderit first so we can log errors as the main tabs are being displayed 
document.getElementsByClassName('logAccordion')[0].parentElement.parentElement.style.order = 100;

// how we talk to python
var kernel = IPython.notebook.kernel;

// log: adds log messages to the logTextarea belonging to the log accordion
function log(msg) {
    kernel.execute("app.loggingHandler.widget.value += 'javascript: " + msg + "'");
}

log('Hello from Javascript!');

<IPython.core.display.Javascript object>

## Splitting the cells files
You may want to put the code above in separate files so that you have a single cell that runs your entire interface. 
This require only a few simple modifications. Potential names for these files are indicated by a comment at the top of each cell.

In order to import the log hander from `log.py` into any other file in our project, we simply add the following two lines:
```bash
import log
logger = log.HANDLER.createLogger(__name__)
```
We would also replace `logger = HANDLER.createLogger(__name__)` with these lines in `interface.py`.

